## Классификация текстов: Spam or Ham

Меняем значения по заданию

In [1]:
test_size = 0.25
random_state = 29
C = 1.4

msg = ["Good day! We would like to offer cheapest Viagra in the world! You can get it at: www.getforfree.com",
       "If you are interested, please write back and I will provide further instructions.",
       "Excellent collection of articles and speeches.",
       "Acceptance of materials for publication: until September 13, 2020. Date of publication and distribution of the collection: until September 20, 2020. ISSN - 2658-7556"]

Запускаем код

In [2]:
import pandas as pd
import nltk
from nltk import stem
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support
import re

df = pd.read_csv('https://courses.openedu.ru/assets/courseware/v1/008dcf9ca6aab621d3b3831db6815867/asset-v1:ITMOUniversity+AUTTEXTAN+fall_2021_ITMO+type@asset+block/3_data.csv', encoding='latin-1')
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df = df.drop_duplicates('text')
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

nltk.download("stopwords", quiet=True)
nltk.download('punkt', quiet=True)
stemmer = SnowballStemmer("english")
stopwords = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    text_tokens = text.split(" ")
    tokens_without_sw = [stemmer.stem(word) for word in text_tokens if not word in stopwords]
    return (" ").join(tokens_without_sw)

df['text'] = df['text'].apply(preprocess)
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=test_size, random_state=random_state)

vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

#обучаем подель SVM

model = LinearSVC(random_state = random_state, C = C)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

precision, recall, f1score, support = precision_recall_fscore_support(y_test, predictions)
print("Precision (macro avg): %.3f" %(sum(precision)/len(precision)))
print("Recall (macro avg): %.3f" %(sum(recall)/len(recall)))
print("F-score (macro avg): %.3f" %(sum(f1score)/len(f1score)))
print()

# Задание 2
print("%s\t%s" %("Предикт", "Сообщение"))
for i in msg:
  print("%d\t%s" %(model.predict(vectorizer.transform([preprocess(i)])[0]), i))

Precision (macro avg): 0.984
Recall (macro avg): 0.936
F-score (macro avg): 0.959

Предикт	Сообщение
0	Good day! We would like to offer cheapest Viagra in the world! You can get it at: www.getforfree.com
0	If you are interested, please write back and I will provide further instructions.
0	Excellent collection of articles and speeches.
0	Acceptance of materials for publication: until September 13, 2020. Date of publication and distribution of the collection: until September 20, 2020. ISSN - 2658-7556
